In [73]:
import os
import re
import csv

In [74]:
units = {
    "s": 1000.0,
    "ms": 1.0,
    "µs": 0.001,
    "ns": 0.000001,
    "bytes": 1.0 / 1048576  # 1 MB = 1024 * 1024 B
}

def removeConsecutiveDuplicates(s):
    if len(s) < 2:
        return s
    if s[0] != s[1] or s[0] != '.':
        return s[0] + removeConsecutiveDuplicates(s[1:])
    return removeConsecutiveDuplicates(s[1:])

def splitString(str):
    alpha = ""
    num = ""
    for i in range(len(str)):
        if (str[i].isdigit() or str[i] == '.'):
            num = num + str[i]
        else:
            alpha += str[i]  
    return alpha, num

In [75]:
# sumcheck
data = []

# Read files
for filename in os.listdir('sumcheck'):
    if filename.startswith('sumcheck_') and filename.endswith('.txt'):
        with open(f'sumcheck/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'd_time(ms)': None, 'd_comm(MB)': None, 'l_time(ms)': None}
            record['n'] = int(re.findall(r'sumcheck_(\d+)', filename)[0])

            for line in content:
                if 'End:' in line and 'SumcheckProduct' in line:
                    cleaned_line = removeConsecutiveDuplicates(line)
                    time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                    if time_string:
                        alpha, num = splitString(time_string.group(0))
                        time_in_ms = float(num) * units[alpha.strip()]
                        if 'Distributed' in line:
                            record['d_time(ms)'] = time_in_ms
                        else:
                            record['l_time(ms)'] = time_in_ms

                if 'Comm:' in line:
                    comm = sum(map(int, re.findall(r'(\d+), (\d+)', line)[0]))
                    record['d_comm(MB)'] = comm * units['bytes']

            data.append(record)

# Save to .csv
csv_file = 'sumcheck.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'd_time(ms)', 'd_comm(MB)', 'l_time(ms)'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)


In [76]:
# poly_comm
data = []

# Read files
for filename in os.listdir('poly_comm'):
    if filename.startswith('poly_comm_') and filename.endswith('.txt'):
        with open(f'poly_comm/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'd_time(ms)': None, 'l_time(ms)': None}
            record['n'] = int(re.findall(r'poly_comm_(\d+)', filename)[0])

            for line in content:
                if 'End:' in line and 'Distributed' in line:
                    cleaned_line = removeConsecutiveDuplicates(line)
                    time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                    if time_string:
                        alpha, num = splitString(time_string.group(0))
                        time_in_ms = float(num) * units[alpha.strip()]
                        record['d_time(ms)'] = time_in_ms
                if 'End:' in line and 'Local' in line:
                    cleaned_line = removeConsecutiveDuplicates(line)
                    time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                    if time_string:
                        alpha, num = splitString(time_string.group(0))
                        time_in_ms = float(num) * units[alpha.strip()]
                        record['l_time(ms)'] = time_in_ms

            data.append(record)

# Save to .csv
csv_file = 'poly_comm.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'd_time(ms)', 'l_time(ms)'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [77]:
# d_Plonk
data = []

d_plonk_operations = [
    ("FFT1", "local"),
    ("Compute r", "local"),
    ("Compute t", "local"),
    ("Local DPP", "local"),
    ("DppRand", "local"),
    ("Division", "local"),
    ("FFT2", "king"),
    ("DPP", "king"),
    ("ComToKing", "comm"),
    ("ComFromKing", "comm"),
    ("Opening shares", "king"),
    ("Packing shares", "king"),
    ("Unpack Pack shares", "king"),
    ("Base MSM", "msm")
]

# Read files
for filename in os.listdir('d_plonk'):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'd_plonk/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'd_time(ms)': 0, 'd_comm(MB)': 0, 'local': 0, 'king': 0, "comm": 0, "msm": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)', filename)[0])
            
            for line in content:
                for op_name, op_var in d_plonk_operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'bytes_sent:' in line:
                    bytes_sent = int(re.search(r'bytes_sent: (\d+)', line).group(1))
                    record['d_comm(MB)'] += bytes_sent * units['bytes']

                if 'bytes_recv:' in line:
                    bytes_recv = int(re.search(r'bytes_recv: (\d+)', line).group(1))
                    record['d_comm(MB)'] += bytes_recv * units['bytes']

             # Calculate total time
            record['d_time(ms)'] = record['local'] + record['king'] + record['comm'] + record['msm']
            
            data.append(record)

# Save to .csv
csv_file = 'd_plonk.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'd_time(ms)', 'd_comm(MB)', 'local','king', "comm", "msm"])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [78]:
# local_GKR Depth=16
data = []

gkr_operations = [
    ("Commit", "poly_comm"),
    ("Initialize phase one", "sumcheck"),
    ("Initialize phase two", "sumcheck"),
    ("Sumcheck product", "sumcheck"),
    ("Calculate f3*f2(u)", "sumcheck"),
    ("Open", "poly_comm"),
]

# Read files
for filename in os.listdir('local_gkr_depth_16'):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'local_gkr_depth_16/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'l_time(ms)': 0, 'poly_comm': 0, "sumcheck": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)', filename)[0])
            
            for line in content:
                for op_name, op_var in gkr_operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

            # Calculate total time
            record['l_time(ms)'] = (record['sumcheck'] + record['poly_comm']) 
            # * 1.2
            
            data.append(record)

# Save to .csv
csv_file = 'l_gkr_depth_16.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'l_time(ms)', 'sumcheck', 'poly_comm'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [79]:
# d_GKR Depth=16
data = []

d_gkr_operations = [
    ("dCommit", "poly_comm"),
    ("dInitialize phase one", "sumcheck"),
    ("dInitialize phase two", "sumcheck"),
    ("dSumcheck product", "sumcheck"),
    ("Calculate f3*f2(u)", "sumcheck"),
    ("dOpen", "poly_comm"),
]

# Read files
for filename in os.listdir('d_gkr_depth_16'):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'd_gkr_depth_16/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'd_time(ms)': 0, 'd_comm(MB)': 0, 'poly_comm': 0, "sumcheck": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)_5', filename)[0])
            
            for line in content:
                for op_name, op_var in d_gkr_operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'Comm:' in line:
                    comm = sum(map(int, re.findall(r'(\d+), (\d+)', line)[0]))
                    record['d_comm(MB)'] = comm * units['bytes']

             # Calculate total time
            record['d_time(ms)'] = record['sumcheck'] + record['poly_comm']
            
            data.append(record)

# Save to .csv
csv_file = 'd_gkr_depth_16.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'd_time(ms)', 'd_comm(MB)', 'sumcheck', 'poly_comm'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [80]:
# d_GKR Depth=16 n=22 varying parties
data = []
n = 19

d_gkr_operations = [
    ("dCommit", "poly_comm"),
    ("dInitialize phase one", "sumcheck"),
    ("dInitialize phase two", "sumcheck"),
    ("dSumcheck product", "sumcheck"),
    ("Calculate f3*f2(u)", "sumcheck"),
    ("dOpen", "poly_comm"),
]

# Read files
for filename in os.listdir('d_gkr_l'):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'd_gkr_l/{filename}', 'r') as file:
            content = file.readlines()

            record = {'parties': None, 'd_time(ms)': 0, 'd_comm(MB)': 0, 'poly_comm': 0, "sumcheck": 0}
            L = int(re.findall(fr'log_0_{n}_(\d+)', filename)[0]) # also K in the paper
            record['parties'] = 2**L * 4 # party = 2^L * 4 where 4 is a magic number from corrupt threshold
            
            for line in content:
                for op_name, op_var in d_gkr_operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'Comm:' in line:
                    comm = sum(map(int, re.findall(r'(\d+), (\d+)', line)[0]))
                    record['d_comm(MB)'] = comm * units['bytes']

             # Calculate total time
            record['d_time(ms)'] = record['sumcheck'] + record['poly_comm']
            
            data.append(record)

# Save to .csv
csv_file = 'd_gkr_parties.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['parties', 'd_time(ms)', 'd_comm(MB)', 'sumcheck', 'poly_comm'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)